# Test Your Algorithm

## Instructions
1. From the **Pulse Rate Algorithm** Notebook you can do one of the following:
   - Copy over all the **Code** section to the following Code block.
   - Download as a Python (`.py`) and copy the code to the following Code block.
2. In the bottom right, click the <span style="color:blue">Test Run</span> button. 

### Didn't Pass
If your code didn't pass the test, go back to the previous Concept or to your local setup and continue iterating on your algorithm and try to bring your training error down before testing again.

### Pass
If your code passes the test, complete the following! You **must** include a screenshot of your code and the Test being **Passed**. Here is what the starter filler code looks like when the test is run and should be similar. A passed test will include in the notebook a green outline plus a box with **Test passed:** and in the Results bar at the bottom the progress bar will be at 100% plus a checkmark with **All cells passed**.
![Example](example.png)

1. Take a screenshot of your code passing the test, make sure it is in the format `.png`. If not a `.png` image, you will have to edit the Markdown render the image after Step 3. Here is an example of what the `passed.png` would look like 
2. Upload the screenshot to the same folder or directory as this jupyter notebook.
3. Rename the screenshot to `passed.png` and it should show up below.
![Passed](passed.png)
4. Download this jupyter notebook as a `.pdf` file. 
5. Continue to Part 2 of the Project. 

In [4]:
import glob
import numpy as np
import scipy as sp
import scipy.io
import scipy.signal as sig
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold,train_test_split 
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor,RandomForestRegressor
from tqdm import tqdm 
import warnings
warnings.filterwarnings("ignore")
def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]



def LoadTroikaRefFile(ref_file):
    data = sp.io.loadmat(ref_file)['BPM0']
    return data

def BandpassFilter(signal, pass_band=(40/60,240/60), fs=125):
      
    b, a = sp.signal.butter(2, pass_band, btype='bandpass', fs = fs)
    return sp.signal.filtfilt(b, a, signal)


def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)




def extract_features(ppg, accx, accy, accz, fs = 125):   
    """
    Create features from the data
    
    Returns:
        PPG and ACC features
    """
    
    mn_x = np.mean(accx)
    std_x = np.std(accx)
    energy_x = np.sum(np.square(accx - np.mean(accx)))
    
    mn_y = np.mean(accy)
    std_y = np.std(accy)
    energy_y = np.sum(np.square(accy - np.mean(accy)))
    
    mn_z = np.mean(accz)
    std_z = np.std(accz)
    energy_z = np.sum(np.square(accz - np.mean(accz)))
                   
    corr_xy = sp.stats.pearsonr(accx, accy)[0]
    
    mn_ppg = np.mean(ppg)
    std_ppg = np.std(ppg)
    
                      
                      
                      
    # magnitude of accelometer
    acc=np.sqrt(accx**2 + accy**2 + accz**2)
    # frequencies and coefficients for both ppg and acceleormeter magnitude 
    fft_acc=np.fft.rfft(acc,6*len(acc))
    freqs_acc=np.fft.rfftfreq(6*len(acc),1/fs)
    fft_ppg=np.fft.rfft(ppg,6*len(ppg))
    freqs_ppg=np.fft.rfftfreq(6*len(ppg),1/fs)
    #bandpass 
    fft_ppg[freqs_ppg <= 40/60] = 0.0
    fft_ppg[freqs_ppg >= 240/60] = 0.0
        
    # getting peaks from the largest frequencies 
    #1) If the ppg peaks does not match with the acc peaks, have that in your result.
    #2) If the ppg peaks match with the acc peaks, 
    #look for the second max peak of ppg.
    #3) Sometimes, the second peak of ppg also match with the acc peak, 
    #in such case have that in your result.
    max_acc_freq=np.argmax(np.abs(fft_acc))
    max_ppg_freq=np.argmax(np.abs(fft_ppg))
    acc_peak=freqs_acc[max_acc_freq]
    ppg_peak=freqs_ppg[max_ppg_freq]
    if acc_peak==ppg_peak:
        fft_ppg=np.delete(fft_ppg,max_ppg_freq)
        freqs_ppg=np.delete(freqs_ppg,max_ppg_freq)
        max_acc_freq=np.argmax(np.abs(fft_acc))
        max_ppg_freq=np.argmax(np.abs(fft_ppg))
        acc_peak=freqs_acc[max_acc_freq]
        ppg_peak=freqs_ppg[max_ppg_freq]
    return np.array([ppg_peak, acc_peak,mn_x,std_x,energy_x,
                    mn_y,std_y,energy_y,mn_z,std_z,energy_z,mn_ppg,std_ppg])




def ensemble_model():       
    """
    Train Regressor model
    
    Returns:
        Trained model
    """ 
    fs = 125
    window_length = 8
    window_shift = 2
    
    # Load filenames through LoadTroikaDataset
    data_fls, ref_fls = LoadTroikaDataset()
    features, labels, signals = [], [], []
    
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
        ref=LoadTroikaRefFile(ref_fl)
        features, labels, signals = [], [], []
        left_s = (np.cumsum(np.ones(min(len(ppg), len(ref))) * fs * window_shift) - fs * window_shift).astype(int)
        right_s = left_s + fs * window_length
        for i in range(len(left_s)):
            left, right = left_s[i], right_s[i]
            ppg_w=BandpassFilter(ppg[left:right])
            accx_w=BandpassFilter(accx[left:right])
            accy_w=BandpassFilter(accy[left:right])
            accz_w=BandpassFilter(accz[left:right])
            features.append(extract_features(ppg_w, accx_w, accy_w, accz_w))
            labels.append(ref[i])
            signals.append([ppg_w, accx_w, accy_w, accz_w])
    
    features, labels = np.array(features), np.array(labels)
    model = RandomForestRegressor(n_estimators=1000)
    model.fit(features,labels)
    
    return model

def RunPulseRateAlgorithm(data_fl, ref_fl):
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    ref=LoadTroikaRefFile(ref_fl)
    print(f" len of ref ={len(ref)}")
    features, labels, signals = [], [], []
    # Load ground truth heart rate
    fs=125
    window_length=8
    window_shift=2 
    left_s = (np.cumsum(np.ones(min(len(ppg), len(ref))) * fs * window_shift) - fs * window_shift).astype(int)
    right_s = left_s + fs * window_length
    for i in range(len(left_s)):
        left, right = left_s[i], right_s[i]
        ppg_w=BandpassFilter(ppg[left:right])
        accx_w=BandpassFilter(accx[left:right])
        accy_w=BandpassFilter(accy[left:right])
        accz_w=BandpassFilter(accz[left:right])
        features.append(extract_features(ppg_w, accx_w, accy_w, accz_w))
        
        labels.append(ref[i])
        signals.append([ppg_w, accx_w, accy_w, accz_w])
    
    features, labels = np.array(features), np.array(labels)
    print(f" len of labels {len(labels)}")
    model= ensemble_model()
    
    # Compute pulse rate estimates and estimation confidence.
    errors, confidence = [], []
    
    for i in range(len(signals)):
        feature, label = features[i], labels[i]
        ppg, accx, accy, accz = signals[i]
        est = model.predict(np.reshape(feature, (1, -1)))[0]
        
        n = len(ppg) * 3
        freqs = np.fft.rfftfreq(n, 1/fs)
        fft = np.abs(np.fft.rfft(ppg,n))
        fft[freqs <= 40/60.0] = 0.0
        fft[freqs >= 240/60.0] = 0.0
          # Max magnitude frequency
        est_fs = est / 55.0
        fs_win = 30  / 60.0
        fs_win_e = (freqs >= est_fs - fs_win) & (freqs <= est_fs +fs_win)
        conf = np.sum(fft[fs_win_e])/np.sum(fft)
        
        errors.append(np.abs((est-labels[i])))
        confidence.append(conf)
    print(f" len of errors ={len(errors),len(confidence)} \n ")

    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
    return np.mean(errors), np.mean(confidence)